# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

Requirement already up-to-date: tensorflow in /home/laura/.local/lib/python3.8/site-packages (2.6.0)
Requirement already up-to-date: keras in /home/laura/.local/lib/python3.8/site-packages (2.6.0)
Requirement already up-to-date: Pillow in /home/laura/.local/lib/python3.8/site-packages (8.3.1)


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=32, activation='elu'))
cnn.add(tf.keras.layers.Dense(units=32, activation='sigmoid'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 95s 374ms/step - loss: 0.6813 - accuracy: 0.5518 - val_loss: 0.6275 - val_accuracy: 0.6325
Epoch 2/25
250/250 [==============================] - 81s 325ms/step - loss: 0.5983 - accuracy: 0.6790 - val_loss: 0.5726 - val_accuracy: 0.7020
Epoch 3/25
250/250 [==============================] - 95s 381ms/step - loss: 0.5534 - accuracy: 0.7134 - val_loss: 0.5244 - val_accuracy: 0.7335
Epoch 4/25
250/250 [==============================] - 102s 408ms/step - loss: 0.5286 - accuracy: 0.7379 - val_loss: 0.5247 - val_accuracy: 0.7450
Epoch 5/25
250/250 [==============================] - 73s 293ms/step - loss: 0.5019 - accuracy: 0.7533 - val_loss: 0.5006 - val_accuracy: 0.7555
Epoch 6/25
250/250 [==============================] - 85s 342ms/step - loss: 0.4839 - accuracy: 0.7632 - val_loss: 0.4855 - val_accuracy: 0.7595
Epoch 7/25
250/250 [==============================] - 88s 352ms/step - loss: 0.4671 - accuracy: 0.7797 - val_loss: 0.4621 - val_a

## Part 4 - Making a single prediction

In [15]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [16]:
print(prediction)

cat
